In [1]:
import tensorflow as tf
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "3"
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.4

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/", one_hot=True)

X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])
keep_prob = tf.placeholder(tf.float32)

X_image = tf.reshape(X, [-1,28,28,1])


/etc/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ./mnist/t10k-images-idx3-ubyte.gz
Extracting ./mnist/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:

####################################################################################
#                             Place your code HERE                                 #
####################################################################################


#Conv1
W1 = tf.Variable(tf.random_normal([5, 5, 1, 128], stddev=0.01))
L1 = tf.nn.relu(tf.nn.conv2d(X_image, filter=W1, strides=[1, 1, 1, 1, ], padding='VALID'))

#Conv2
W2 = tf.Variable(tf.random_normal([5, 5, 128, 64], stddev=0.01))
L2 = tf.nn.relu(tf.nn.conv2d(L1, filter=W2, strides=[1, 1, 1, 1, ], padding='VALID'))

#Pool1
L2 = tf.nn.max_pool(L2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

#Conv3
W3 = tf.Variable(tf.random_normal([1, 1, 64, 64], stddev=0.01))
L3 = tf.nn.relu(tf.nn.conv2d(L2, filter=W3, strides=[1, 1, 1, 1, ], padding='SAME'))

#Conv4
W4 = tf.Variable(tf.random_normal([1, 1, 64, 32], stddev=0.01))
L4 = tf.nn.relu(tf.nn.conv2d(L3, filter=W4, strides=[1, 1, 1, 1, ], padding='SAME'))

#Pool2
L4 = tf.nn.max_pool(L4, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

#Reshape
L4 = tf.reshape(L4, [-1, 5 * 5 * 32])

#Dense1
W5 = tf.Variable(tf.random_normal([5 * 5 * 32, 256], stddev=0.01))
B5 = tf.Variable(tf.random_normal(shape=[256], stddev=0.01))
L5 = tf.nn.relu(tf.matmul(L4, W5) + B5)

#Dence2
W6 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
B6 = tf.Variable(tf.random_normal(shape=[10], stddev=0.01))
model = tf.nn.softmax(tf.matmul(L5, W6) + B6)

#Cost, Optimizer
cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(tf.clip_by_value(model, 1e-10, 1.0)), [1]))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

init = tf.global_variables_initializer()
sess = tf.Session(config=config)
sess.run(init)

batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

for epoch in range(15):
    total_cost = 0

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)

        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys, keep_prob: 0.7})
        total_cost += cost_val

    print('Epoch:', '%04d' % (epoch + 1), 'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))

print('Training Done!')
print('Test Acc. = ', sess.run(accuracy, feed_dict={X: mnist.test.images[0:10], Y: mnist.test.labels[0:10], keep_prob: 1.0}))
sess.close()

Epoch: 0001 Avg. cost = 0.359
Epoch: 0002 Avg. cost = 0.105
Epoch: 0003 Avg. cost = 0.079
Epoch: 0004 Avg. cost = 0.063
Epoch: 0005 Avg. cost = 0.051
Epoch: 0006 Avg. cost = 0.044
Epoch: 0007 Avg. cost = 0.039
Epoch: 0008 Avg. cost = 0.035
Epoch: 0009 Avg. cost = 0.030
Epoch: 0010 Avg. cost = 0.027
Epoch: 0011 Avg. cost = 0.024
Epoch: 0012 Avg. cost = 0.021
Epoch: 0013 Avg. cost = 0.019
Epoch: 0014 Avg. cost = 0.017
Epoch: 0015 Avg. cost = 0.015
Training Done!
Test Acc. =  1.0
